In [44]:
import pandas as pd
import numpy as np
import pdfplumber
import re
import nltk
from scraping import Scraping
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# cleaning the function

In [2]:
def extract_clean(self):
        file = pdfplumber.open(self.pdf)
        if len(self.page) > 1 :
            joined_text = []
            for el in self.page:
                page = file.pages[el]
                text = page.extract_text()
                joined_text.append(text)
            text ='             NEXT          '.join(joined_text)
            text = text.replace('Isento', '0,00')
            text = text.replace('n/a', str(np.nan))
            text = re.sub('--', str(np.nan), text)
            if nltk.sent_tokenize(text) < 15:
                text = text.replace('\n','. ')
                # text = len_sentences(text)
                text = nltk.sent_tokenize(text)
                return text
            text = text.replace('\n',' ')
            # text = len_sentences(text)
            text = nltk.sent_tokenize(text)
            return text
        else:
            page = file.pages[self.page[0]]
            text = page.extract_text()
            text = text.replace('Isento', '0,00')
            text = text.replace('n/a', str(np.nan))
            text = re.sub('--', str(np.nan), text)
            if nltk.sent_tokenize(text) < 15:
                text = text.replace('\n','. ')
                # text = len_sentences(text)
                text = nltk.sent_tokenize(text)
                return text
            text = text.replace('\n',' ')
            # text = len_sentences(text)
            text = nltk.sent_tokenize(text)
            return text

In [3]:
ctt = pdfplumber.open('data/BancoCTT2021.pdf')
page = ctt.pages[4]
text = page.extract_text()
len(nltk.sent_tokenize(text))

4

In [4]:
bankinter = pdfplumber.open('data/bankinter.pdf')
page = bankinter.pages[4]
text = page.extract_text()
len(nltk.sent_tokenize(text))

40

In [5]:
bai = pdfplumber.open('data/bancoBAIeurope2021com.pdf')
page = bai.pages[4]
text = page.extract_text()
len(nltk.sent_tokenize(text))

23

In [13]:
Scraping('data/bancoBAIeurope2021com.pdf', 4, '4').extract_clean()

['Entrada em vigor: 25-jan-2021 1.',
 'CONTAS DE DEPÓSITO (PARTICULARES) (ÍNDICE) 1.1.',
 'Depósitos à ordem Comissões Acresce  Outras condições  (Mín/Máx) Valor Anual Imposto Conta Depósito à Ordem (EUR/ Moeda Estrangeira) Comissões vigência contrato  5,00 €/ Moeda Estrangeira  60,00 €/ Moeda  1.',
 'Manutenção de conta I.S.',
 '- 4% Nota (1) por Mês Estrangeira 2.',
 'Levantamento de numerário 20,00 € 0,5% Incide sobre o montante do  3.',
 'Levantamento USD em contas USD I.S.',
 '- 4% (25,00 €/ 275,00 €) levantamento.',
 '4.',
 'Extracto integrado 0,00 Periodicidade: Mensal  5,00 €/ Moeda Estrangeira  5.',
 'Extracto avulso I.S.',
 '- 4% A pedido do cliente.',
 'por cada Comissões Descoberto Bancário  vide Secção 2.5 Descobertos bancários Conta de Serviços Mínimos Bancários Comissões vigência contrato 1.',
 'Manutenção de conta 4,10 €/por Ano 4,10 € I.S.',
 '- 4% Nota (2) Comissões Descoberto Bancário  vide Secção 2.5 Descobertos bancários Outras despesas associadas nan.',
 'Legenda:

# connecting name of product and name of commission

In [39]:
abanca = Scraping('data/abanca.pdf', 4, 'abanca')
ctt = Scraping('data/ctt.pdf',4,'5')
bai = Scraping('data/bai.pdf',4,'4')
bic = Scraping('data/bic.pdf',5,'3')
bankinter = Scraping('data/bankinter.pdf',4,'2')

In [55]:
ab_t = abanca.tokenize()
ab_n = abanca.names()

In [50]:
ab_n

['Conta D.O.',
 'Conta Ordenado',
 'Conta Standard',
 'Conta Future',
 'Conta Kids',
 'Conta Global Em comercialização:',
 'Conta Base',
 'Conta Private',
 'Conta Value',
 'Conta Smart',
 'Conta Futuro',
 'Conta Serviços Mínimos Bancários,',
 'Conta Moeda Estrangeira e',
 'Conta ABANCA Internacional.',
 'Conta D.O.',
 'Conta Ordenado (Nota 1 e Nota 16) 0,00 nan 3.3',
 'Conta Standard (não admite novas contratações) 0,00 nan 3.4',
 'Conta Future (Nota 3) 0,00 nan 3.5',
 'Conta Kids (Nota 4) 0,00 nan IS – 4% 3.6',
 'Conta Global (Nota 5) 6,53 / 7,23 Mensal 85,14 3.7',
 'Conta Base (Nota 6) 10,00 Mensal 120,00 3.8',
 'Conta Private (Nota 7) 8,00 Mensal 96,00 3.9',
 'Conta Value (Nota 8) 5,00 Mensal 60,00 3.10',
 'Conta Smart (Nota 9) 5,00 Mensal 60,00 3.11',
 'Conta Futuro (Nota 10) 0,00 nan 3.12',
 'Conta Serviços Mínimos Bancários (Nota 11) 2,00 Semestral 4,00 3.13',
 'Conta Moeda Estrangeira (Nota 12) 5,00 Mensal 60,00 3.14',
 'Conta ABANCA Internacional (Nota 17) 0,00 nan 4.',
 'Conta

In [53]:
com_dict.values()
    

dict_values([['Emissão de Extrato', 'Extrato Integrado', 'Extrato Mensal'], ['Fotocópias de segundas vias de talões de depósito', 'Emissão 2ªs Vias de Avisos e Outros Documentos', 'Extracto avulso', 'Segundas vias (pedido na agência)'], ['Manutenção de conta', 'Comissão de manutenção de conta'], ['Levantamento de numerário', 'Levantamento de numerário ao balcão'], ['Adesão ao serviço de banca à distância', 'Adesão ao serviço online'], ['Depósito de moedas metálicas', 'Depósito de moedas', 'Depósito de moedas ao balcão', 'Depósito de dinheiro ao balcão', 'Depósito de moeda metálica (≥ a 100 moedas)'], ['Alteração de titulares', 'Alteração de titularidade', 'Alteração de titularidade / intervenientes'], ['Comissões por descoberto bancário', 'Descoberto bancário'], ['Consulta de Movimentos de conta DO com', 'Consulta de movimentos ao balcão'], ['Pedido de saldo ao balcão', 'Consulta de Saldo de conta DO com comprovativo']])

In [52]:
com_dict = {'Emissão de Extrato':['Emissão de extrato', 'Extrato Integrado', 'Extrato Mensal'],
           'Fotocópias e 2ªvias':['Fotocópias de segundas vias de talões de depósito',
                                  'Emissão 2ªs Vias de Avisos e Outros Documentos', 'Extracto avulso',
                                 'Segundas vias (pedido na agência)'],
           'Manutenção de Conta':['Manutenção de conta', 'Comissão de manutenção de conta'],
           'Levantamento de Numerário':['Levantamento de numerário', 'Levantamento de numerário ao balcão'],
           'Adesão ao Serviço Online':['Adesão ao serviço de banca à distância', 'Adesão ao serviço online'],
            'Depósitos de Moedas':['Depósito de moedas metálicas', 'Depósito de moedas',
                                   'Depósito de moedas ao balcão', 'Depósito de dinheiro ao balcão',
                                  'Depósito de moeda metálica (≥ a 100 moedas)'],
            'Ateração de Titulares':['Alteração de titulares', 'Alteração de titularidade',
                                     'Alteração de titularidade / intervenientes'],
            'Descoberto Bancário':['Comissões por descoberto bancário', 'Descoberto bancário'],
            'Consulta de Movimentos':['Consulta de Movimentos de conta DO com', 'Consulta de movimentos ao balcão'],
            'Consulta de Saldo':['Pedido de saldo ao balcão', 'Consulta de Saldo de conta DO com comprovativo']
           }

In [56]:
ab_t[:5]

['Entrada em vigor: 20-Nov-2020    1.',
 'CONTAS DE DEPÓSITO (PARTICULARES)                                                                                   ÍNDICE   1.1 Depósitos à ordem  Comissões  Acresce  Outras    Euros (Mín/Máx)  Valor anual  Imposto  condições  Produtos: Fora de comercialização: Conta D.O.',
 'Particulares; Conta Ordenado; Conta Standard; Conta Future; Conta  Kids; Conta Global  Em comercialização: Conta Base; Conta Private; Conta Value; Conta Smart; Conta Futuro; Conta Serviços Mínimos  Bancários, Conta Moeda Estrangeira e Conta ABANCA Internacional.',
 'As comissões abaixo indicadas ocorrem no momento em que se verifica a operação e aplicam-se às contas acima identificadas, salvo  nos casos em que expressamente se exclui.',
 '1.']

In [68]:
# keys = [x for x in com_dict.keys()]
values = [x for x in com_dict.values()]
lista ={}
for commission in values:
    for value in commission:
        for ind,sentence in enumerate(ab_t):
            if value in sentence:
                if value in lista:
                    if '[0-9]{1-2},[0-9]{2}' in sentence:
                        lista[value]= lista[value].append(sentence)
                    else:
                        lista[value]= lista[value].append(' '.join([sentence,file[ind+1]]))
                else:
                    if '[0-9]{1-2},[0-9]{2}' in sentence:
                        lista[value]= [sentence]
                    else:
                        lista[value]= [' '.join([sentence,ab_t[ind+1]])]
# return lista
                

In [69]:
len(lista)

7

In [71]:
abanca.values()

{'Emissão de extrato': ['Emissão de extrato      1.1  Mensal (enviado ao domicílio)  0,00  nan  IVA – 23%  1.2  Outros, para além do indicado em 1.1  2,00 por extrato  1.3  2ª Via      7,50    2. Fotocópias de segundas vias de talões de depósito   5,00 por documento  nan  IVA – 23%    3.'],
 'Fotocópias de segundas vias de talões de depósito': ['Fotocópias de segundas vias de talões de depósito   5,00 por documento  nan  IVA – 23%    3. Manutenção de conta      3.1 Conta D.O.'],
 'Manutenção de conta': ['Manutenção de conta      3.1 Conta D.O. Particulares (Nota 2)      - Saldo médio até 1.000€  15,00 Trimestral  60,00  - Saldo médio entre 1.001€ e 2.500€  10,00 Trimestral  40,00  - Saldo médio superior a 2.500€  0,00  nan  3.2 Conta Ordenado (Nota 1 e Nota 16)  0,00  nan  3.3 Conta Standard (não admite novas contratações)  0,00  nan  3.4 Conta Future (Nota 3)  0,00  nan  3.5 Conta Kids (Nota 4)  0,00  nan  IS – 4%    3.6 Conta Global (Nota 5)  6,53 / 7,23 Mensal  85,14  3.7 Conta Base

# how to join and search for the name of commision and account together

In [ ]:
com_dict = {'Emissão de Extrato':['Emissão de extrato', 'Extrato Integrado', 'Extrato Mensal'],
           'Fotocópias e 2ªvias':['Fotocópias de segundas vias de talões de depósito',
                                  'Emissão 2ªs Vias de Avisos e Outros Documentos', 'Extracto avulso',
                                 'Segundas vias (pedido na agência)'],
           'Manutenção de Conta':['Manutenção de conta', 'Comissão de manutenção de conta'],
           'Levantamento de Numerário':['Levantamento de numerário', 'Levantamento de numerário ao balcão'],
           'Adesão ao Serviço Online':['Adesão ao serviço de banca à distância', 'Adesão ao serviço online'],
            'Depósitos de Moedas':['Depósito de moedas metálicas', 'Depósito de moedas',
                                   'Depósito de moedas ao balcão', 'Depósito de dinheiro ao balcão',
                                  'Depósito de moeda metálica (≥ a 100 moedas)'],
            'Ateração de Titulares':['Alteração de titulares', 'Alteração de titularidade',
                                     'Alteração de titularidade / intervenientes'],
            'Descoberto Bancário':['Comissões por descoberto bancário', 'Descoberto bancário'],
            'Consulta de Movimentos':['Consulta de Movimentos de conta DO com', 'Consulta de movimentos ao balcão'],
            'Consulta de Saldo':['Pedido de saldo ao balcão', 'Consulta de Saldo de conta DO com comprovativo']
           }

In [86]:
ab_t = abanca.tokenize()
ab_n = abanca.names() 

In [93]:
results = {}
type_account = {}
for name in ab_n: 
    for key in com_dict:
        for values in key:
            print('object')
            for value in values:
                for ind,sentence in enumerate(ab_t):
                    if name in sentence and value in sentence[:ind+3]:
                        if name in type_account:
                            type_account[name].append(sentence)
                        else:
                            type_account[name]=[sentence]
                            print(type_account)

object
object
object
object
object
object
{'Conta D.O.': ['Manutenção de conta      3.1 Conta D.O.']}
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
ob

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object

In [90]:
type_account

{'Conta D.O.': ['Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O.',
  'Manutenção de conta      3.1 Conta D.O

In [82]:
for ind,sentence in enumerate(ab_t):
    for name in ab_n:
        if name in sentence:
            results[name]=sentence

In [85]:
ab_n

['Conta D.O.',
 'Conta Ordenado',
 'Conta Standard',
 'Conta Future',
 'Conta Kids',
 'Conta Global Em comercialização:',
 'Conta Base',
 'Conta Private',
 'Conta Value',
 'Conta Smart',
 'Conta Futuro',
 'Conta Serviços Mínimos Bancários,',
 'Conta Moeda Estrangeira e',
 'Conta ABANCA Internacional.',
 'Conta D.O.',
 'Conta Ordenado (Nota 1 e Nota 16) 0,00 nan 3.3',
 'Conta Standard (não admite novas contratações) 0,00 nan 3.4',
 'Conta Future (Nota 3) 0,00 nan 3.5',
 'Conta Kids (Nota 4) 0,00 nan IS – 4% 3.6',
 'Conta Global (Nota 5) 6,53 / 7,23 Mensal 85,14 3.7',
 'Conta Base (Nota 6) 10,00 Mensal 120,00 3.8',
 'Conta Private (Nota 7) 8,00 Mensal 96,00 3.9',
 'Conta Value (Nota 8) 5,00 Mensal 60,00 3.10',
 'Conta Smart (Nota 9) 5,00 Mensal 60,00 3.11',
 'Conta Futuro (Nota 10) 0,00 nan 3.12',
 'Conta Serviços Mínimos Bancários (Nota 11) 2,00 Semestral 4,00 3.13',
 'Conta Moeda Estrangeira (Nota 12) 5,00 Mensal 60,00 3.14',
 'Conta ABANCA Internacional (Nota 17) 0,00 nan 4.',
 'Conta

In [84]:
ab_t

'Entrada em vigor: 20-Nov-2020 \n \n1.  CONTAS DE DEPÓSITO (PARTICULARES)                                                                                   ÍNDICE  \n1.1 Depósitos à ordem  Comissões  Acresce  Outras \n  Euros (Mín/Máx)  Valor anual  Imposto  condições \nProdutos: Fora de comercialização: Conta D.O. Particulares; Conta Ordenado; Conta Standard; Conta Future; Conta \nKids; Conta Global \nEm comercialização: Conta Base; Conta Private; Conta Value; Conta Smart; Conta Futuro; Conta Serviços Mínimos \nBancários, Conta Moeda Estrangeira e Conta ABANCA Internacional. \nAs comissões abaixo indicadas ocorrem no momento em que se verifica a operação e aplicam-se às contas acima identificadas, salvo \nnos casos em que expressamente se exclui.  \n1.  Emissão de extrato     \n1.1  Mensal (enviado ao domicílio)  0,00 \nnan  IVA – 23% \n1.2  Outros, para além do indicado em 1.1  2,00 por extrato \n1.3  2ª Via      7,50   \n2.   Fotocópias de segundas vias de talões de depósito   5,00 

In [96]:
Scraping('data/2_all_products.pdf', 4, '7').getting_text()

TypeError: list indices must be integers or slices, not list